# Import libraries

In [1]:
#import tensorflow.keras as keras
import numpy as np
import pandas as pd
import sklearn
from joblib import dump, load

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score, auc, precision_recall_curve, accuracy_score

from tensorflow import keras
from tensorflow.keras import optimizers, losses, activations, models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D, \
    concatenate,SimpleRNN,LSTM,Embedding,GRU,Bidirectional,Masking
import tensorflow
tensorflow.random.set_seed(36)

# Import data & data marshalling
## Create first beat plot

In [2]:
ptbdb_normal = pd.read_csv("ptbdb_normal.csv", header=None)
ptbdb_abnormal = pd.read_csv("ptbdb_abnormal.csv", header=None)
ptbdb = pd.concat([ptbdb_normal, ptbdb_abnormal])

mitbih_train = pd.read_csv("mitbih_train.csv", header=None)
mitbih_train = mitbih_train.sample(frac=1)
mitbih_test = pd.read_csv("mitbih_test.csv", header=None)


ptbdb_categories=2
ptbdb_train, ptbdb_test = train_test_split(ptbdb, test_size=0.2, random_state=42, stratify=ptbdb[187])
ptbdb_train_Y = np.array(ptbdb_train[187].values).astype(np.int8)
ptbdb_train_X = np.array(ptbdb_train[list(range(187))].values)[..., np.newaxis].squeeze()
ptbdb_validation_x, ptbdb_train_x, ptbdb_validation_y, ptbdb_train_y = train_test_split(ptbdb_train_X, ptbdb_train_Y, test_size=0.33, random_state=42)
ptbdb_test_Y = np.array(ptbdb_test[187].values).astype(np.int8)
ptbdb_test_X = np.array(ptbdb_test[list(range(187))].values)[..., np.newaxis].squeeze()
ptbdb_beats=range(len(ptbdb_train_x))
ptbdb_color=['green','red']
ptbdb_label=["Normal beat", "Abnormal beat"]

mitbih_categories=5
mitbih_train_Y = np.array(mitbih_train[187].values).astype(np.int8)
mitbih_train_X = np.array(mitbih_train[list(range(187))].values)[..., np.newaxis].squeeze()
mitbih_test_Y = np.array(mitbih_test[187].values).astype(np.int8)
mitbih_test_X = np.array(mitbih_test[list(range(187))].values)[..., np.newaxis].squeeze()
mitbih_validation_x, mitbih_train_x, mitbih_validation_y, mitbih_train_y = train_test_split(mitbih_train_X, mitbih_train_Y, test_size=0.33, random_state=42)
mitbih_beats=range(len(mitbih_train_x))
mitbih_color=['green','red','black','blue','grey']
mitbih_label=["Normal", "Supraventricular", "Premature","Fusion", "Unclassifiable"]

for category in range(ptbdb_categories):
    for beat in ptbdb_beats:
        if ptbdb_train_y[beat]==category:
            plt.plot(range(len(ptbdb_train_x[beat])),ptbdb_train_x[beat],color=ptbdb_color[category],label=ptbdb_label[category])
            break
plt.legend()
plt.xlabel("time")
plt.ylabel("intensity")
plt.title('PTBDB Different beats')
plt.savefig("ptbdb_dif_beats.png")
plt.clf()

for category in range(mitbih_categories):
    for beat in mitbih_beats:
        if mitbih_train_y[beat]==category:
            plt.plot(range(len(mitbih_train_x[beat])),mitbih_train_x[beat],color=mitbih_color[category],label=mitbih_label[category])
            break
plt.xlabel("time")
plt.ylabel("intensity")
plt.title('MITBIH Different beats')
plt.legend()
plt.savefig("mitbih_dif_beats.png")
plt.clf()

<Figure size 432x288 with 0 Axes>

# Create peaks and peaks plot

In [3]:
window=15
def make_peaks(data):
    peaks=np.zeros((len(data),2*window))
    for i in range(len(data)):
        #I started after 10 because sometimes there is a big peak at the very start
        peak=np.argmax(data[i][10:])+10
        peakinfo=data[i][peak-window:peak+window]
        for j in range(len(peakinfo)):
            peaks[i][j]=peakinfo[j]
    return pd.DataFrame(peaks)

ptbdb_peaks=make_peaks(ptbdb_train_X)
ptbdb_cat_peaks=pd.concat([ptbdb_peaks, pd.DataFrame(ptbdb_train_Y).rename(columns={0: "Category"})], axis=1)

mitbih_peaks=make_peaks(mitbih_train_X)
mitbih_cat_peaks=pd.concat([mitbih_peaks, pd.DataFrame(mitbih_train_Y).rename(columns={0: "Category"})], axis=1)

ptbdb_cat_peaks=ptbdb_cat_peaks.groupby(['Category']).mean().T
mitbih_cat_peaks=mitbih_cat_peaks.groupby(['Category']).mean().T

for category in range(ptbdb_categories):
    plt.plot(range(len(ptbdb_cat_peaks)),ptbdb_cat_peaks[category],color=ptbdb_color[category],label=ptbdb_label[category])
plt.xlabel("time")
plt.ylabel("intensity")
plt.title('PTBDB Mean peaks')
plt.legend()
plt.savefig("ptbdb_peaks.png")
plt.clf()

for category in range(mitbih_categories):
    plt.plot(range(len(mitbih_cat_peaks)),mitbih_cat_peaks[category],color=mitbih_color[category],label=mitbih_label[category])
plt.xlabel("time")
plt.ylabel("intensity")
plt.title('MITBIH Mean peaks')
plt.legend()
plt.savefig("mitbih_peaks.png")
plt.clf()

<Figure size 432x288 with 0 Axes>

# Make distribution plot /how many of each category exist

In [4]:
ptbdb_counts=ptbdb[187].value_counts(ascending=True).sort_index()
mitbih_counts=pd.Series(mitbih_train_Y).value_counts().sort_index()

for category in range(ptbdb_categories):
    plt.bar(category,ptbdb_counts[category],color=ptbdb_color[category],label=ptbdb_label[category])
plt.ylabel("Amount")
plt.xticks([])
plt.title('PTBDB Beat distribution')
plt.legend()
plt.savefig("ptbdb_distribution.png")
plt.clf()

for category in range(mitbih_categories):
    plt.bar(category,mitbih_counts[category],color=mitbih_color[category],label=mitbih_label[category])
plt.ylabel("Amount")
plt.xticks([])
plt.title('MITBIH Beat distribution')
plt.legend()
plt.savefig("mitbih_distribution.png")
plt.clf()

<Figure size 432x288 with 0 Axes>

# Make differences and create differences histogram

In [5]:
def make_differences(X):
    differences=np.zeros((len(X),len(X[1])))
    signals=np.zeros(len(X))
    for beat in range(len(X)):
        for signal in range(1,len(X[1])):
            if X[beat][signal-1]==0 and X[beat][signal]==0:
                if (signal>=len(X[1])-2):
                    signals[beat]=signal
                    break
                else:
                    if X[beat][signal+1]==0 and X[beat][signal+2]==0:
                        signals[beat]=signal
                        break
            differences[beat][signal]=X[beat][signal]-X[beat][signal-1]
    return differences,signals

ptbdb_difCat=[[],[]]
ptbdb_differences, ptbdb_signals=make_differences(ptbdb_train_X)
for beat in range(len(ptbdb_differences)):
    ptbdb_difCat[ptbdb_train_Y[beat]].extend(ptbdb_differences[beat][:int(ptbdb_signals[beat]-1)])

X=mitbih_train_X

mitbih_difCat=[[],[],[],[],[]]
mitbih_differences, mitbih_signals=make_differences(mitbih_train_X)
for beat in range(len(mitbih_differences)):
    mitbih_difCat[mitbih_train_Y[beat]].extend(mitbih_differences[beat][:int(mitbih_signals[beat]-1)])


plt.hist(ptbdb_difCat, bins=[-1/20,-1/40,-1/80,-1/160,0,1/160,1/80,1/40,1/20], density=True,color=ptbdb_color, label=ptbdb_label)
plt.xticks([])
plt.title('PTBDB difference histogram inner')
plt.legend()
plt.savefig("ptbdb_histo_inner.png")
plt.clf()

plt.hist(ptbdb_difCat,bins=[-1,-1/2,-1/4,-1/10,-1/20,-1/40],density=True,color=ptbdb_color, label=ptbdb_label)
plt.xticks([])
plt.title('PTBDB difference histogram left')
plt.legend()
plt.savefig("ptbdb_histo_left.png")
plt.clf()

plt.hist(ptbdb_difCat,bins=[1/40,1/20,1/10,1/4,1/2,1],density=True,color=ptbdb_color, label=ptbdb_label)
plt.xticks([])
plt.title('PTBDB difference histogram right')
plt.legend()
plt.savefig("ptbdb_histo_right.png")
plt.clf()

plt.hist(mitbih_difCat, bins=[-1/20,-1/40,-1/80,-1/160,0,1/160,1/80,1/40,1/20], density=True,color=mitbih_color, label=mitbih_label)
plt.xticks([])
plt.title('MITBIH difference histogram inner')
plt.legend()
plt.savefig("mitbih_histo_inner.png")
plt.clf()

plt.hist(mitbih_difCat,bins=[-1,-1/2,-1/4,-1/10,-1/20,-1/40],density=True,color=mitbih_color, label=mitbih_label)
plt.xticks([])
plt.title('MITBIH difference histogram left')
plt.legend()
plt.savefig("mitbih_histo_left.png")
plt.clf()

plt.hist(mitbih_difCat,bins=[1/40,1/20,1/10,1/4,1/2,1],density=True,color=mitbih_color, label=mitbih_label)
plt.xticks([])
plt.title('MITBIH difference histogram right')
plt.legend()
plt.savefig("mitb_histo_right.png")
plt.clf()

<Figure size 432x288 with 0 Axes>

# PTBDB baseline

In [6]:
df_1 = pd.read_csv("ptbdb_normal.csv", header=None)

df_2 = pd.read_csv("ptbdb_abnormal.csv", header=None)

df = pd.concat([df_1, df_2])

df_train, df_test = train_test_split(df, test_size=0.2, random_state=1337, stratify=df[187])

Y = np.array(df_train[187].values).astype(np.int8)
X = np.array(df_train[list(range(187))].values)[..., np.newaxis]

Y_test = np.array(df_test[187].values).astype(np.int8)
X_test = np.array(df_test[list(range(187))].values)[..., np.newaxis]


def get_model():
    nclass = 1
    inp = Input(shape=(187, 1))
    img_1 = Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid")(inp)
    img_1 = Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = GlobalMaxPool1D()(img_1)
    img_1 = Dropout(rate=0.2)(img_1)

    dense_1 = Dense(64, activation=activations.relu, name="dense_1")(img_1)
    dense_1 = Dense(64, activation=activations.relu, name="dense_2")(dense_1)
    dense_1 = Dense(nclass, activation=activations.sigmoid, name="dense_3_ptbdb")(dense_1)

    model = models.Model(inputs=inp, outputs=dense_1)
    opt = optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=losses.binary_crossentropy, metrics=['acc'])
    model.summary()
    return model

model = get_model()
file_path = "baseline_cnn_ptbdb.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early

model.fit(X, Y, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
model.load_weights(file_path)

pred_test = model.predict(X_test)
test_probs = pred_test
pred_test = (pred_test>0.5).astype(np.int8)

ptbdb_base_acc = accuracy_score(Y_test, pred_test)

ptbdb_base_auroc = roc_auc_score(Y_test, test_probs)

precision, recall, thresh = precision_recall_curve(Y_test,test_probs)
ptbdb_base_auprc=auc(recall,precision)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 187, 1)]          0         
_________________________________________________________________
conv1d (Conv1D)              (None, 183, 16)           96        
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 179, 16)           1296      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 89, 16)            0         
_________________________________________________________________
dropout (Dropout)            (None, 89, 16)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 87, 32)            1568      
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 85, 32)            3104  

# MITBIH baseline

In [7]:
df_train = pd.read_csv("mitbih_train.csv", header=None)
df_train = df_train.sample(frac=1)
df_test = pd.read_csv("mitbih_test.csv", header=None)

Y = np.array(df_train[187].values).astype(np.int8)
X = np.array(df_train[list(range(187))].values)[..., np.newaxis]

Y_test = np.array(df_test[187].values).astype(np.int8)
X_test = np.array(df_test[list(range(187))].values)[..., np.newaxis]

def get_model():
    nclass = 5
    inp = Input(shape=(187, 1))
    img_1 = Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid")(inp)
    img_1 = Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = GlobalMaxPool1D()(img_1)
    img_1 = Dropout(rate=0.2)(img_1)

    dense_1 = Dense(64, activation=activations.relu, name="dense_1")(img_1)
    dense_1 = Dense(64, activation=activations.relu, name="dense_2")(dense_1)
    dense_1 = Dense(nclass, activation=activations.softmax, name="dense_3_mitbih")(dense_1)

    model = models.Model(inputs=inp, outputs=dense_1)
    opt = optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=losses.sparse_categorical_crossentropy, metrics=['acc'])
    model.summary()
    return model

model = get_model()
file_path = "baseline_cnn_mitbih.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early

model.fit(X, Y, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
model.load_weights(file_path)

pred_test = model.predict(X_test)
test_probs = pred_test
pred_test = np.argmax(pred_test, axis=-1)


mitbih_base_acc = accuracy_score(Y_test, pred_test)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 187, 1)]          0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 183, 16)           96        
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 179, 16)           1296      
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 89, 16)            0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 89, 16)            0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 87, 32)            1568      
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 85, 32)            3104

# own RNNs
## Hyperparameters based on results from 'Healthcare_GPU.ipynb'

In [8]:
def ptbdb_scores(model,X,Y):
    pred_test = model.predict(X)
    test_probs = pred_test
    pred_test = np.argmax(pred_test, axis=-1)
    acc = accuracy_score(Y, pred_test)

    auroc = roc_auc_score(Y, test_probs)

    precision, recall, thresh = precision_recall_curve(Y,test_probs)
    auprc=auc(recall,precision)
    return acc,auroc,auprc
early = EarlyStopping(monitor="val_accuracy", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_accuracy", mode="max", patience=3, verbose=2)
callbacks_list = [early, redonplat]  # early

def build(input_size, num_classes, gate_type,dropout_rate,recurrent_dropout_rate,units,num_layers,optimizer_type,learning_rate):
    model = keras.Sequential()
    model.add(keras.layers.Masking(mask_value=0.,
                              input_shape=(input_size, 1)))
    for i in range(num_layers):
        if gate_type=='SimpleRNN':
            model.add(SimpleRNN(units,return_sequences=True,dropout=dropout_rate,recurrent_dropout=recurrent_dropout_rate))
        elif gate_type=='GRU':
            model.add(GRU(units,return_sequences=True,dropout=dropout_rate,recurrent_dropout=recurrent_dropout_rate))
        elif gate_type=='LSTM':
            model.add(LSTM(units,return_sequences=True,dropout=dropout_rate,recurrent_dropout=recurrent_dropout_rate))
        elif gate_type=='Bidirectional':
            model.add(Bidirectional(LSTM(units,return_sequences=True,dropout=dropout_rate,recurrent_dropout=recurrent_dropout_rate)))
    if gate_type=='SimpleRNN':
        model.add(SimpleRNN(units,dropout=dropout_rate,recurrent_dropout=recurrent_dropout_rate))
    elif gate_type=='GRU':
        model.add(GRU(units,dropout=dropout_rate,recurrent_dropout=recurrent_dropout_rate))
    elif gate_type=='LSTM':
        model.add(LSTM(units,dropout=dropout_rate,recurrent_dropout=recurrent_dropout_rate))
    elif gate_type=='Bidirectional':
        model.add(Bidirectional(LSTM(units,dropout=dropout_rate,recurrent_dropout=recurrent_dropout_rate)))
    model.add(Dense(64, activation=activations.relu, name="dense_1"))
    model.add(Dense(64, activation=activations.relu, name="dense_2"))
    model.add(Dense(num_classes-1, activation=activations.softmax, name="dense_3_mitbih"))
    if optimizer_type=='SGD':
        optimizer=keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer_type=='ADAM':
        optimizer=keras.optimizers.Adam(
            learning_rate=learning_rate)
    elif optimizer_type=='RMSprop':
        optimizer=keras.optimizers.RMSprop(
            learning_rate=learning_rate)
    if num_classes >2:
        loss=losses.sparse_categorical_crossentropy
    else:
        loss=losses.binary_crossentropy
    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=['accuracy'])
    return model

# Hyperparameters: PTBDB normal/diff
# |-dropout: 0.30000000000000004
# |-gate_type: GRU
# |-learning_rate: 0.01
# |-num_layers: 1
# |-optimizer: ADAM
# |-recurrent_dropout: 0.2
# |-tuner/bracket: 3
# |-tuner/epochs: 5
# |-tuner/initial_epoch: 2
# |-tuner/round: 1
# |-tuner/trial_id: 1be35566db3f7b602560eef664260300
# |-units: 137
ptbdb_normal_model=build(187,2, 'GRU',0.3,0.2,137,1,'ADAM',0.01)
ptbdb_diff_model=build(187,2, 'GRU',0.3,0.2,137,1,'ADAM',0.01)

# Hyperparameters: MITBIH normal/diff
# |-dropout: 0.2
# |-gate_type: SimpleRNN
# |-learning_rate: 0.01
# |-num_layers: 1
# |-optimizer: ADAM
# |-recurrent_dropout: 0.1
# |-tuner/bracket: 3
# |-tuner/epochs: 2
# |-tuner/initial_epoch: 0
# |-tuner/round: 0
# |-units: 168
mitbih_normal_model=build(187,5,'SimpleRNN',0.2,0.1,168,1,'ADAM',0.01)
mitbih_diff_model=build(187,5,'SimpleRNN',0.2,0.1,168,1,'ADAM',0.01)

# Hyperparameters: PTBDB/MITBIH peaks
# |-dropout: 0.2
# |-gate_type: SimpleRNN
# |-learning_rate: 0.01
# |-num_layers: 1
# |-optimizer: ADAM
# |-recurrent_dropout: 0.1
# |-tuner/bracket: 3
# |-tuner/epochs: 2
# |-tuner/initial_epoch: 0
# |-tuner/round: 0
# |-units: 24
ptbdb_peaks_model=build(30,2,'SimpleRNN',0.2,0.1,24,1,'ADAM',0.01)
mitbih_peaks_model=build(30,5,'SimpleRNN',0.2,0.1,24,1,'ADAM',0.01)


ptbdb_train_peaks=np.expand_dims(make_peaks(ptbdb_train_X),2)
ptbdb_test_peaks=np.expand_dims(make_peaks(ptbdb_test_X),2)
mitbih_train_peaks=np.expand_dims(make_peaks(mitbih_train_X),2)
mitbih_test_peaks=np.expand_dims(make_peaks(mitbih_test_X),2)

ptbdb_train_diff=np.expand_dims(make_differences(ptbdb_train_X)[0],2)
ptbdb_test_diff=np.expand_dims(make_differences(ptbdb_test_X)[0],2)
mitbih_train_diff=np.expand_dims(make_differences(mitbih_train_X)[0],2)
mitbih_test_diff=np.expand_dims(make_differences(mitbih_test_X)[0],2)

ptbdb_normal_model.fit(np.expand_dims(ptbdb_train_X,2),ptbdb_train_Y,epochs=1000, verbose=1, callbacks=callbacks_list, validation_split=0.1)
ptbdb_peaks_model.fit(ptbdb_train_peaks,ptbdb_train_Y,epochs=1000, verbose=1, callbacks=callbacks_list, validation_split=0.1)
ptbdb_diff_model.fit(ptbdb_train_diff,ptbdb_train_Y,epochs=1000, verbose=1, callbacks=callbacks_list, validation_split=0.1)

mitbih_normal_model.fit(np.expand_dims(mitbih_train_X,2),mitbih_train_Y,epochs=1000, verbose=1, callbacks=callbacks_list, validation_split=0.1)
mitbih_peaks_model.fit(mitbih_train_peaks,mitbih_train_Y,epochs=1000, verbose=1, callbacks=callbacks_list, validation_split=0.1)
mitbih_diff_model.fit(mitbih_train_diff,mitbih_train_Y,epochs=1000, verbose=1, callbacks=callbacks_list, validation_split=0.1)


ptbdb_normal_acc,ptbdb_normal_auroc,ptbdb_normal_auprc=ptbdb_scores(ptbdb_normal_model,np.expand_dims(ptbdb_test_X,2),ptbdb_test_Y)
ptbdb_peaks_acc,ptbdb_peaks_auroc,ptbdb_peaks_auprc=ptbdb_scores(ptbdb_peaks_model,ptbdb_test_peaks,ptbdb_test_Y)
ptbdb_diff_acc,ptbdb_diff_auroc,ptbdb_diff_auprc=ptbdb_scores(ptbdb_diff_model,ptbdb_test_diff,ptbdb_test_Y)

pred_test = mitbih_normal_model.predict(np.expand_dims(mitbih_test_X,2))
pred_test = np.argmax(pred_test, axis=-1)
mitbih_normal_acc = accuracy_score(mitbih_test_Y, pred_test)

pred_test = mitbih_peaks_model.predict(mitbih_test_peaks)
pred_test = np.argmax(pred_test, axis=-1)
mitbih_peaks_acc = accuracy_score(mitbih_test_Y, pred_test)

pred_test = mitbih_diff_model.predict(mitbih_test_diff)
pred_test = np.argmax(pred_test, axis=-1)
mitbih_diff_acc = accuracy_score(mitbih_test_Y, pred_test)

Train on 10476 samples, validate on 1165 samples
Epoch 1/1000
10476/10476 [==============================] - 224s 21ms/sample - loss: 4.2577 - accuracy: 0.7208 - val_loss: 4.0839 - val_accuracy: 0.7322
Epoch 2/1000
10476/10476 [==============================] - 219s 21ms/sample - loss: 4.2577 - accuracy: 0.7208 - val_loss: 4.0839 - val_accuracy: 0.7322
Epoch 3/1000
10476/10476 [==============================] - 219s 21ms/sample - loss: 4.2577 - accuracy: 0.7208 - val_loss: 4.0839 - val_accuracy: 0.7322
Epoch 4/1000
10464/10476 [============================>.] - ETA: 0s - loss: 4.2597 - accuracy: 0.7207
Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0009999999776482583.
10476/10476 [==============================] - 220s 21ms/sample - loss: 4.2577 - accuracy: 0.7208 - val_loss: 4.0839 - val_accuracy: 0.7322
Epoch 5/1000
10476/10476 [==============================] - 219s 21ms/sample - loss: 4.2577 - accuracy: 0.7208 - val_loss: 4.0839 - val_accuracy: 0.7322
Epoch 6/1000
1047

# own SVCs
## Hyperparameters based on 'Healthcare.ipynb'

In [11]:
mitbih_clf_normal = load('mitbih_clf_normal.joblib')
#I did not have enough time to hypertune the other svc's (the two I did took more than a day), the hyperparameters for these svc's correspond to the ones found in the hypertuned svc's.
print(mitbih_clf_normal.best_params_)
mitbih_clf_normal=sklearn.svm.SVC(random_state= 36, kernel='rbf', C=10,probability=True)
mitbih_clf_diff=sklearn.svm.SVC(random_state= 36, kernel='rbf', C=10,probability=True)
ptbdb_clf_normal=sklearn.svm.SVC(random_state= 36, kernel='rbf', C=10,probability=True)
ptbdb_clf_diff=sklearn.svm.SVC(random_state= 36, kernel='rbf', C=10,probability=True)
mitbih_clf_peaks = load('mitbih_clf_peaks.joblib')
print(mitbih_clf_peaks.best_params_)
mitbih_clf_peaks =sklearn.svm.SVC(random_state= 36, kernel='rbf', C=10,probability=True)
ptbdb_clf_peaks =sklearn.svm.SVC(random_state= 36, kernel='rbf', C=10,probability=True)

mitbih_clf_normal.fit(mitbih_train_X,mitbih_train_Y)
mitbih_clf_peaks.fit(mitbih_train_peaks.squeeze(), mitbih_train_Y)
mitbih_clf_diff.fit(mitbih_train_diff.squeeze(),mitbih_train_Y)

ptbdb_clf_normal.fit(ptbdb_train_X,ptbdb_train_Y)
ptbdb_clf_peaks.fit(ptbdb_train_peaks.squeeze(),ptbdb_train_Y)
ptbdb_clf_diff.fit(ptbdb_train_diff.squeeze(),ptbdb_train_Y)

ptbdb_svc_normal_acc=ptbdb_clf_normal.score(ptbdb_test_X,ptbdb_test_Y)
ptbdb_svc_peaks_acc=ptbdb_clf_peaks.score(ptbdb_test_peaks.squeeze(),ptbdb_test_Y)
ptbdb_svc_diff_acc=ptbdb_clf_diff.score(ptbdb_test_diff.squeeze(),ptbdb_test_Y)

test_probs_normal=ptbdb_clf_normal.predict_proba(ptbdb_test_X)[:,1]
test_probs_peaks=ptbdb_clf_peaks.predict_proba(ptbdb_test_peaks.squeeze())[:,1]
test_probs_diff=ptbdb_clf_diff.predict_proba(ptbdb_test_diff.squeeze())[:,1]

ptbdb_svc_normal_auroc=roc_auc_score(ptbdb_test_Y, test_probs_normal)
ptbdb_svc_peaks_auroc = roc_auc_score(ptbdb_test_Y, test_probs_peaks)
ptbdb_svc_diff_auroc = roc_auc_score(ptbdb_test_Y, test_probs_diff)

precision, recall, thresh = precision_recall_curve(ptbdb_test_Y,test_probs_normal)
ptbdb_svc_normal_auprc=auc(recall,precision)
precision, recall, thresh = precision_recall_curve(ptbdb_test_Y,test_probs_peaks)
ptbdb_svc_peaks_auprc=auc(recall,precision)
precision, recall, thresh = precision_recall_curve(ptbdb_test_Y,test_probs_diff)
ptbdb_svc_diff_auprc=auc(recall,precision)

mitbih_svc_normal_acc = mitbih_clf_normal.score(mitbih_test_X,mitbih_test_Y)
mitbih_svc_peaks_acc = mitbih_clf_peaks.score(mitbih_test_peaks.squeeze(),mitbih_test_Y)
mitbih_svc_diff_acc = mitbih_clf_diff.score(mitbih_test_diff.squeeze(),mitbih_test_Y)

C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator SVC from version 0.21.3 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator RandomizedSearchCV from version 0.21.3 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


{'random_state': 36, 'kernel': 'rbf', 'class_weight': None, 'C': 10}
{'random_state': 36, 'kernel': 'rbf', 'class_weight': None, 'C': 10}


# Transfer learning
## imports from models created at 'Transferral_learning.ipynb'

In [12]:
transfer_base=tensorflow.keras.models.load_model('transferral_base.h5')
transfer2=tensorflow.keras.models.load_model('transfer2.h5')
transfer2_acc,transfer2_auroc,transfer2_auprc=ptbdb_scores(transfer2,np.expand_dims(transfer_base.predict(np.expand_dims(ptbdb_test_X,2)),2),ptbdb_test_Y)

transfer4 = tensorflow.keras.models.load_model('transfer4.h5')
transfer4_acc,transfer4_auroc,transfer4_auprc=ptbdb_scores(transfer4,np.expand_dims(ptbdb_test_X,2),ptbdb_test_Y)
transfer3 = tensorflow.keras.models.load_model('transfer3.h5')
transfer3_acc,transfer3_auroc,transfer3_auprc=ptbdb_scores(transfer3,np.expand_dims(ptbdb_test_X,2),ptbdb_test_Y)

# Results

In [15]:
print('PTBDB Baseline')
print('Normal: '+str(ptbdb_base_acc)+', '+str(ptbdb_base_auroc)+', '+str(ptbdb_base_auprc))

print('PTBDB RNN')
print('Normal: '+str(ptbdb_normal_acc)+', '+str(ptbdb_normal_auroc)+', '+str(ptbdb_normal_auprc))
print('Peaks: '+str(ptbdb_peaks_acc)+', '+str(ptbdb_peaks_auroc)+', '+str(ptbdb_peaks_auprc))
print('Diff: '+str(ptbdb_diff_acc)+', '+str(ptbdb_diff_auroc)+', '+str(ptbdb_diff_auprc))

print('PTBDB SVC')
print('Normal: '+str(ptbdb_svc_normal_acc)+', '+str(ptbdb_svc_normal_auroc)+', '+str(ptbdb_svc_normal_auprc))
print('Peaks: '+str(ptbdb_svc_peaks_acc)+', '+str(ptbdb_svc_peaks_auroc)+', '+str(ptbdb_svc_peaks_auprc))
print('Diff: '+str(ptbdb_svc_diff_acc)+', '+str(ptbdb_svc_diff_auroc)+', '+str(ptbdb_svc_diff_auprc))

print('PTBDB Transfer Learning')
print('Transfer 2: '+str(transfer2_acc)+', '+str(transfer2_auroc)+', '+str(transfer2_auprc))
print('Transfer 3: '+str(transfer3_acc)+', '+str(transfer3_auroc)+', '+str(transfer3_auprc))
print('Transfer 4: '+str(transfer4_acc)+', '+str(transfer4_auroc)+', '+str(transfer4_auprc))


print('MITBIH Baseline')
print('Normal: '+str(mitbih_base_acc))

print('MITBIH RNN')
print('Normal: '+str(mitbih_normal_acc))
print('Peaks: '+str(mitbih_peaks_acc))
print('Diff: '+str(mitbih_diff_acc))

print('MITBIH SVC')
print('Normal: '+str(mitbih_svc_normal_acc))
print('Peaks: '+str(mitbih_svc_peaks_acc))
print('Diff: '+str(mitbih_svc_diff_acc))

PTBDB Baseline
Normal: 0.9893507385778083, 0.9983308027318735, 0.9992689170662377
PTBDB RNN
Normal: 0.2779113706630024, 0.5, 0.8610443146684987
Peaks: 0.2779113706630024, 0.5, 0.8610443146684987
Diff: 0.2779113706630024, 0.5, 0.8610443146684987
PTBDB SVC
Normal: 0.9529371350051529, 0.9786612079378166, 0.9886240971271106
Peaks: 0.9148059086224665, 0.9516173895248388, 0.9766256606219932
Diff: 0.9495018893850911, 0.9783965826883337, 0.9900106687736172
PTBDB Transfer Learning
Transfer 2: 0.2779113706630024, 0.5, 0.8610443146684987
Transfer 3: 0.2779113706630024, 0.5, 0.8610443146684987
Transfer 4: 0.2779113706630024, 0.5, 0.8610443146684987
MITBIH Baseline
Normal: 0.9834186004019734
MITBIH RNN
Normal: 0.8276082587246483
Peaks: 0.8276082587246483
Diff: 0.8276082587246483
MITBIH SVC
Normal: 0.9774346793349169
Peaks: 0.9513064133016627
Diff: 0.9766581399598027
